In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from ta.trend import WMAIndicator
from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange
from ta.volume import AccDistIndexIndicator
from ta.utils import IndicatorMixin

class AccDistOscillatorIndicator(IndicatorMixin):
    def __init__(
        self,
        high: pd.Series,
        low: pd.Series,
        closeY: pd.Series,
        fillna: bool = False,
    ):
        
        self._high = high
        self._low = low
        self._closeY = closeY
        self._fillna = fillna
        self._run()
        
    def _run(self):
        ado = (self._high - self._closeY)/(self._high - self._low)
        self._ado = ado
        
    def acc_dist_oscillator(self) -> pd.Series:
        ado = self._check_fillna(self._ado, value=0)
        return pd.Series(ado, name="ado")

def acc_dist_oscillator(high, low, closeY, fillna=False):
    return AccDistOscillatorIndicator(high=high, low=low, closeY=closeY, fillna=fillna).acc_dist_oscillator()
    
def ticker_data_processing(ticker, time_period):
    tick = yf.download(ticker, period=time_period)
    tickerData = tick
    #adjusting the Adj Close columns 
    tickerData = tickerData.drop(['Adj Close'], axis =1)
    close = tickerData['Close']
    tickerData['Today Close'] = close
    tickerData['Close'] = tickerData['Close'].shift(-1)
    tickerData = tickerData.rename(columns={"Close" : "Tomorrow Close"})
    tickerData['Yesterday Close'] = tickerData['Today Close'].shift(+1)
    tickerData = tickerData.dropna()
    
    #using TA library to calculate and add TA data to the data set
    indicator_wma = WMAIndicator(close=tickerData["Today Close"], window = 10)
    tickerData['WMA'] = indicator_wma.wma()
    
    indicator_rsi = RSIIndicator(close=tickerData["Today Close"], window = 10)
    tickerData['RSI'] = indicator_rsi.rsi()
    
    indicator_atr = AverageTrueRange(high=tickerData["High"], low=tickerData["Low"], close=tickerData["Today Close"], window = 10)
    tickerData['ATR'] = indicator_atr.average_true_range()
    
    indicator_sma = SMAIndicator(close=tickerData["Today Close"], window = 10)
    tickerData['SMA'] = indicator_sma.sma_indicator()
    
    indicator_ado = AccDistOscillatorIndicator(high=tickerData["High"], low=tickerData["Low"], closeY=tickerData["Yesterday Close"])
    tickerData['ADO'] = indicator_ado.acc_dist_oscillator()
    
    tickerData = tickerData.drop(tickerData.index[:9])
   
    #print(tickerData) ##########################################################################
    
    #graphing the TA indicators
    #fig, grph = plt.subplots(5, figsize =(20, 50))
    #grph[0].plot(tickerData['WMA'])
    #grph[1].plot(tickerData['RSI'])
    #grph[2].plot(tickerData['ATR'])
    #grph[3].plot(tickerData['SMA'])
    #grph[4].plot(tickerData['ADO'])
    
    #drops N/A
    tickerData = tickerData.dropna()
    tickerData = tickerData.drop(['Volume', 'Open', 'High', 'Low', 'Today Close'], axis=1)
    
    #splitting the test and train data and dropping volume from both
    train_set, test_set = train_test_split(tickerData, test_size=0.3, shuffle=False)
    
    #setting up the scaler
    scale = ['WMA', 'RSI', 'ADO', 'ATR', 'SMA']
    scaler = MinMaxScaler()
    
    #seperate features and labels
    train_set_features = train_set.drop(["Tomorrow Close"], axis=1) 
    train_set_labels = train_set['Tomorrow Close'].copy()
    
    test_set_features = test_set.drop(['Tomorrow Close'], axis=1)
    test_set_labels = test_set['Tomorrow Close'].copy()
    
    #scaling the features
    feature_transform = scaler.fit_transform(train_set_features[scale])
    train_prepared = pd.DataFrame(columns=scale, data=feature_transform, index=train_set_features.index)
    
    feature_transform = scaler.transform(test_set_features[scale])
    test_prepared = pd.DataFrame(columns=scale, data=feature_transform, index=test_set_features.index)
    
    
    return train_prepared, train_set_labels, test_prepared, test_set_labels 

ticker_data_processing('F', '5y')

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_percentage_error

def svr_linear(ticker, period):
    train_prepared, train_set_labels, test_prepared, test_set_labels = ticker_data_processing(ticker, period)
    plt.close()
    svm_reg = SVR(kernel="linear", epsilon= .5, C=34058.19149792883, max_iter=2500)
    svm_reg.fit(train_prepared, train_set_labels)
    
    predictions = svm_reg.predict(test_prepared)
    
    scores = cross_val_score(svm_reg, train_prepared, train_set_labels, scoring="neg_mean_squared_error", cv=5 )
    
    rmse = np.sqrt(-scores)
    print("Scores:", rmse)
    print("Mean:", rmse.mean())
    print ("Standared deviation:", rmse.std())
    print("MAPE: ", mean_absolute_percentage_error(test_set_labels, predictions))
    
    
    test_prepared['Predicted Close'] = predictions.tolist()
    test_prepared['Predicted Close'].plot(label="Predicted", figsize=(24,12))
    test_set_labels.plot(label="Close")
    plt.legend()
    
svr_linear('F', '15y')

In [ ]:
from sklearn.svm import SVR

def svr_polynomial(ticker, period):
    train_prepared, train_set_labels, test_prepared, test_set_labels = ticker_data_processing(ticker, period)
    plt.close()
    svm_reg = SVR(kernel= 'poly', epsilon= .5, C=2, degree=1, gamma=10, max_iter=2500)
    svm_reg.fit(train_prepared, train_set_labels)
    
    predictions = svm_reg.predict(test_prepared)
    
    scores = cross_val_score(svm_reg, train_prepared, train_set_labels, scoring="neg_mean_squared_error", cv=5 )
    
    rmse = np.sqrt(-scores)
    print("Scores:", rmse)
    print("Mean:", rmse.mean())
    print ("Standared deviation:", rmse.std())
    print("MAPE: ", mean_absolute_percentage_error(test_set_labels, predictions))
    
    
    test_prepared['Predicted Close'] = predictions.tolist()
    test_prepared['Predicted Close'].plot(label="Predicted", figsize=(24,12))
    test_set_labels.plot(label="Close")
    plt.legend()
    
svr_polynomial('F', '15y')

In [ ]:
def svr_rbf(ticker, period):
    train_prepared, train_set_labels, test_prepared, test_set_labels = ticker_data_processing(ticker, period)
    plt.close()
    svm_reg = SVR(kernel= 'rbf', epsilon= .5, C=2.0, gamma=0.10292942782029627, max_iter=2500)
    svm_reg.fit(train_prepared, train_set_labels)
    
    predictions = svm_reg.predict(test_prepared)
    
    scores = cross_val_score(svm_reg, train_prepared, train_set_labels, scoring="neg_mean_squared_error", cv=5 )
    
    rmse = np.sqrt(-scores)
    print("Scores:", rmse)
    print("Mean:", rmse.mean())
    print ("Standared deviation:", rmse.std())
    print("MAPE: ", mean_absolute_percentage_error(test_set_labels, predictions))
    
    
    test_prepared['Predicted Close'] = predictions.tolist()
    test_prepared['Predicted Close'].plot(label="Predicted", figsize=(24,12))
    test_set_labels.plot(label="Close")
    plt.legend()
    
svr_rbf('F', '15y')

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

def randomWalk(ticker, period):
    plt.close()
    tick = yf.download(ticker, period=period)
    tickerData = tick

    returns = tickerData['Close'].pct_change() #calculates the daily returns of the stock as a percentage
    daily_volatility = returns.std() #calculates the standard deviation of the stock returns to represent sigma in random walk eq.
    
    index = tickerData.index
    length = len(index)
    count = 0
    predicted_prices = []
    last_price = tickerData['Close'][0]

    price = last_price #sets the initial price
    predicted_prices.append(price) 
    
    for y in range(length): #for loop runs for the length of time given
        if count == length-1:
            break
        price = predicted_prices[count] * (1 + np.random.normal(0, daily_volatility)) #calculates prices with random walk equation
        predicted_prices.append(price) #adds price to list
        count += 1
        
    actualPrice = tickerData.Close 
    rmse = mean_squared_error(actualPrice, predicted_prices, squared = False)
    print("MAPE: ", mean_absolute_percentage_error(actualPrice, predicted_prices))
    print("RMSE: ", rmse)
    
    df = pd.DataFrame(predicted_prices)
    df.index = tickerData.index
    
    ax = df.plot()
    tickerData['Close'].plot(label="Close", ax=ax,figsize=(15,7),grid=True)
    
    
randomWalk('HL', '1y')

In [ ]:
import time 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective, plot_histogram

linear = BayesSearchCV(
    SVR(),
    {
        'C': Real(0.1, 2.0, 'log-uniform'),
        'kernel': ['linear'],  # categorical parameter
    },
    n_iter=200,
    n_jobs=4,
    n_points=2,
    cv=2,
    random_state = 1
)

rbf = BayesSearchCV(
    SVR(),
    {
        'C': Real(0.1, 2.0, 'log-uniform'),
        'gamma': Real(0.1, 10, 'log-uniform'),
        'kernel': ['rbf'],  # categorical parameter
    },
    n_iter=200,
    n_jobs=6,
    n_points=3,
    cv=2,
    random_state = 1
)
poly = BayesSearchCV(
    SVR(),
    {
        'C': Real(0.1, 2.0, 'log-uniform'),
        'gamma': Real(0.1, 10, 'log-uniform'),
        'degree': Integer(1, 5, 'log-uniform'),
        'kernel': ['poly'],  # categorical parameter
    },
    n_iter=200,
    n_jobs=8,
    n_points=4,
    cv=2,
    random_state = 1
)
train_x, train_y, test_x, test_y = ticker_data_processing('F', '15y')
t0 = time.time()
model = poly.fit(train_x, train_y)
print(poly.score(test_x, test_y))
print("Parameters:")
print(poly.best_params_)
t1 = time.time()
total = t1-t0
print("It took this long: %s" % total)
